In [1]:
import numpy as np
from numpy import newaxis
import pandas as pd
import matplotlib.pyplot as plt
import datetime
import time
import seaborn as sns
from scipy import stats
from sklearn.model_selection import train_test_split
from sklearn.metrics import explained_variance_score
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import make_scorer
from sklearn.preprocessing import MinMaxScaler
from subprocess import check_output
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.recurrent import LSTM
from keras.models import Sequential


In [2]:
from google.colab import files
uploaded = files.upload()

Saving XRP-USD.csv to XRP-USD.csv


In [ ]:
data = pd.read_csv(r'dataset.csv')
data = pd.read_csv("dataset.csv")
data = data[['Latitude', 'Longitude', 'Depth', 'Magnitude']]
data.head()
data.describe()
scaler = MinMaxScaler()
scaled_data = pd.DataFrame(scaler.fit_transform(data), columns=data.columns)
scaled_data.head()


,Latitude,Longitude,Depth,Magnitude
0,0.590649,0.904493,0.189274,0.138889
1,0.484060,0.853759,0.115675,0.083333
2,0.346451,0.016736,0.030096,0.194444
3,0.110396,0.434562,0.022964,0.083333
4,0.545838,0.851190,0.022964,0.083333


In [ ]:
scaler = MinMaxScaler()
scaled_data = pd.DataFrame(scaler.fit_transform(data), columns=data.columns)
scaled_data.head()

,Latitude,Longitude,Depth,Magnitude
0,0.590649,0.904493,0.189274,0.138889
1,0.484060,0.853759,0.115675,0.083333
2,0.346451,0.016736,0.030096,0.194444
3,0.110396,0.434562,0.022964,0.083333
4,0.545838,0.851190,0.022964,0.083333


In [ ]:
print(len(data))
train_size = int(len(scaled_data) * 0.80)
test_size = len(scaled_data) - train_size
train, test = scaled_data[0:train_size], scaled_data[train_size:len(scaled_data)]
print(len(train), len(test))

23412
18729 4683


In [ ]:
# convert an array of values into a dataset matrix
def generate_dataset(dataset, look_back=1):
	dataX, dataY = [], []
    
	for i in range(len(dataset)-look_back-1):
		a = dataset[i:(i+look_back)]
		dataX.append(np.array(a))
		dataY.append(np.array(dataset[i + look_back:i+look_back+1]))
        
	return np.array(dataX),np.array(dataY)

In [ ]:
#look_back = 50
look_back = 100
trainX, trainY = generate_dataset(train, look_back)
testX, testY = generate_dataset(test, look_back)
print(trainX.shape)
print(trainY.shape)
print(testX.shape)
print(testY.shape)

(18628, 100, 4)
(18628, 1, 4)
(4582, 100, 4)
(4582, 1, 4)


In [ ]:
#Step 2 Build Model
model = Sequential()
model.reset_states()

model.add(LSTM(100, input_shape = (100,4), activation = 'sigmoid', return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(100, activation = 'sigmoid', return_sequences=True))
model.add(Dropout(0.2))
model.add(Dense(4, activation='linear'))
model.compile(loss='mse', optimizer='adam')


In [ ]:
model.fit(trainX, trainY, batch_size=500, epochs=1, validation_split=0.1)
model.summary()

34/34 [==============================] - 80s 2s/step - loss: 0.1310 - val_loss: 0.0460
Model: "sequential_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_28 (LSTM)               (None, 100, 100)          42000     
_________________________________________________________________
dropout_28 (Dropout)         (None, 100, 100)          0         
_________________________________________________________________
lstm_29 (LSTM)               (None, 100, 100)          80400     
_________________________________________________________________
dropout_29 (Dropout)         (None, 100, 100)          0         
_________________________________________________________________
dense_14 (Dense)             (None, 100, 4)            404       
Total params: 122,804
Trainable params: 122,804
Non-trainable params: 0
_________________________________________________________________


In [ ]:
print(testY.shape)
#print(np.array(testX[0:1][0:1]))
testX.shape
mag_test = []
for inner in testX[0]:
    mag_test.append(inner[3])
print(len(mag_test))
depth_test = []
for inner in testX[0]:
    depth_test.append(inner[2])
print(len(depth_test))

(4582, 1, 4)
100
100


In [ ]:
test_pred = model.predict(testX)

In [ ]:
#print(test_pred.shape)
#print(test_pred[0][2])
mag_pred = []
for inner in test_pred[0]:
    mag_pred.append(inner[3])
#print(mag_pred)
depth_pred = []
for inner in test_pred[0]:
    depth_pred.append(inner[2])
print(len(mag_pred))
print(len(depth_pred))

100
100


In [ ]:
train_score_1 = r2_score(mag_test, mag_pred)
train_score_2 = explained_variance_score(mag_test, mag_pred)
print(train_score_1)
print(train_score_2)
print(r2_score(depth_test, depth_pred))
print(explained_variance_score(depth_test, depth_pred))
print('train_score_1 = -0.09985716273213437 -- r2score')
print('train_score_2=0.0024514766668029286 -- explained_variance_score') 
print((mean_squared_error(mag_test, mag_pred)))
print((mean_squared_error(depth_test, depth_pred)))


-0.021537243154923003
0.022478539271054254
-0.32220425982593404
-0.05969207281678113
train_score_1 = -0.09985716273213437 -- r2score
train_score_2=0.0024514766668029286 -- explained_variance_score
0.01454271769769161
0.011724678220315723


You need to multiply your scaled mse by $(max - min)^2$